# This notebook is to take apart video feed and turn each frame into an image that then can be run in MegaDetector.

In [2]:
!pip install opencv-python-headless 

# Keep this just incase an issue happens.


In [3]:
import os
import cv2
import json
from megadetector.utils import path_utils
from megadetector.detection import video_utils
from megadetector.visualization import visualization_utils as vis_utils

# Add the directory containing video_utils.py to the system path
import sys
sys.path.append("C:/Users/Public/Documents/MegaDetector_App/MegaDetector/megadetector/detection")

from video_utils import video_folder_to_frames, frames_to_video, get_video_fs, frame_results_to_video_results
import tkinter as tk
from tkinter import filedialog as fd 

# Create a hidden root window
root = tk.Tk()
root.withdraw()
root.attributes('-topmost', True)  # Ensure it stays on top
# Configure paths
# input_folder = r"C:/Users/Public/Documents/Data/video_data/"
# output_folder_base = r"C:/Users/Public/Documents/Data/video_data/frames/"
# detected_frame_folder_base = r"C:/Users/Public/Documents/Data/video_data_detected/"
# rendered_videos_folder_base = r"C:/Users/Public/Documents/Data/video_data_rendered/"
# json_path = r"C:/Users/Public/Documents/jsons/tests.json"




''

In [4]:
#The following 5 cells each run the same function, Ask directory, they are split up into seperate cells to avoid confusion when choosing folders
# the Ask open file name function allows you to manually select the folder that you are using each time.
#the print function will print the folder name, if it isnt the desired folder simply run the cell again the chose a different one

In [15]:
input_folder = fd.askdirectory()
print("Selected folder:", input_folder)

Selected folder: C:/Users/Public/WLF_Cam_Data/7July2025/BG_SW_Arm_2_7July2025


In [35]:
output_folder_base = fd.askdirectory() # change name to frame folder
print("Selected folder:",output_folder_base)

Selected folder: C:/Users/Public/WLF_Cam_Data/7July2025/Proccessed_Frames


In [19]:
detected_frame_folder_base = fd.askdirectory() # detections folder change name
print("Selected folder:", detected_frame_folder_base)

Selected folder: C:/Users/Public/WLF_Cam_Data/7July2025/BG_SW_Arm_2_7July2025/DCIM/frame_folder_base


In [20]:
rendered_videos_folder_base = fd.askdirectory() # rename 
print("Selected folder:", rendered_videos_folder_base)

Selected folder: C:/Users/Public/WLF_Cam_Data/7July2025/BG_SW_Arm_2_7July2025/DCIM/rendered_videos_folder_base


In [21]:
json_path = fd.askdirectory()
print("Selected folder:", json_path)

Selected folder: C:/Users/Public/WLF_Cam_Data/7July2025/BG_SW_Arm_2_7July2025/DCIM


In [36]:
# Ensure output directories exist
os.makedirs(detected_frame_folder_base, exist_ok=True)
os.makedirs(rendered_videos_folder_base, exist_ok=True)

In [37]:
#From Don Morris Mega detector examples,Managing a local mega detector video batch Notebook
quality = 90 #controls image output quality, lower number smaller files higher number clearer images (out of 100)
max_width = 1600 #controls Frame size, resizes to a max width, avoids huge image files
recursive = True #looks through sub folders,if changes to false only the outer most folders will be processed (must be true to do folders within folders)
overwrite = True #Replaces existing frames (duplicates), if changed to false and duplicates exist extraction might fail or skip (DO NOT TOUCH)
parallelization_uses_threads = True #Speeds up the processing
n_workers = 8 #Controls how many threads are used

# Sample every Nth frame.  To specify a sampling rate in seconds, use a negative
# value.  For example:
#
# * Setting every_n_frames to -2.0 yields a frame rate of 0.5 fps
# * Setting every_n_frames to -0.5 yields a frame rate of 2.0 fps
#
every_n_frames = 10 #Controls how often a frame is extracted, (every 10 frames), without this too few or too many frames may be extracted (DO NOT DELETE)

In [38]:
# Split videos into frames
# Ensure folders exist
assert os.path.isdir(input_folder)
os.makedirs(output_folder_base, exist_ok=True)

# Frame extraction
frame_filenames_by_video, fs_by_video, video_filenames = \
    video_folder_to_frames(
        input_folder=input_folder,
        output_folder_base=output_folder_base,
        recursive=recursive,
        overwrite=overwrite,
        n_threads=n_workers,
        every_n_frames=every_n_frames,
        parallelization_uses_threads=parallelization_uses_threads,
        quality=quality,
        max_width=max_width,
        #allow_empty_videos=True
    )

Found 116 videos in folder C:/Users/Public/WLF_Cam_Data/7July2025/BG_SW_Arm_2_7July2025
Starting a worker pool with 8 threads


100%|████████████████████████████████████████████████████████████████████████████████| 116/116 [00:31<00:00,  3.67it/s]


In [39]:
frame_folder_base = output_folder_base

In [40]:
# Each leaf-node folder *should* correspond to a video; we're going to verify that below.

from collections import defaultdict

frame_files = path_utils.find_images(frame_folder_base,recursive=True)
frame_files = [s.replace('\\','/') for s in frame_files]
print('Enumerated {} total frames'.format(len(frame_files)))

# Find unique (relative) folders
folder_to_frame_files = defaultdict(list)

# fn = frame_files[0]
for fn in frame_files:
    folder_name = os.path.dirname(fn)
    folder_name = os.path.relpath(folder_name,frame_folder_base)
    folder_to_frame_files[folder_name].append(fn)

print('Found {} folders for {} files'.format(len(folder_to_frame_files),len(frame_files)))

Enumerated 3571 total frames
Found 116 folders for 3571 files


In [41]:
video_filenames = video_utils.find_videos(input_folder,recursive=True)
video_filenames = [os.path.relpath(fn,input_folder) for fn in video_filenames]
video_filenames = [fn.replace('\\','/') for fn in video_filenames]
print('Input folder contains {} videos'.format(len(video_filenames)))

Input folder contains 116 videos


116 of 116 folders are missing frames entirely


In [55]:
# 1. Run MegaDetector on frames (shell command)
!python "C:/Users/Public/Documents/MegaDetector_App/MegaDetector/megadetector/detection/run_detector_batch.py" \
    MDV5A "{output_folder_base}" "{json_path}" --output_relative_filenames --recursive --checkpoint_frequency 1000 --quiet

Bypassing download of already-downloaded file md_v5a.0.0.pt
307 image files found in the input directory
The checkpoint file will be written to C:/Users/Public/Documents/jsons/videos_for_MD\md_checkpoint_20250312175206.json
PyTorch reports 1 available CUDA devices
GPU available: True
Imported YOLOv5 as utils.*
Using PyTorch version 2.2.2
Sending model to GPU
Loaded model in 3.31 seconds
Loaded model in 3.31 seconds
Finished inference for 307 images in 11.53 seconds (26.62 images per second)
Output file saved at C:/Users/Public/Documents/jsons/videos_for_MD/RR_ringtail.json
Done, thanks for MegaDetect'ing!


Fusing layers... 
Model summary: 574 layers, 139990096 parameters, 0 gradients, 207.9 GFLOPs

100%|##########| 307/307 [00:07<00:00, 42.53it/s]


In [ ]:
# 2. Aggregate frame-level results into video-level results
video_output_filename = json_path.replace(".json","_aggregated.json")
options = FrameToVideoOptions()
options.include_all_processed_frames = True
frame_results_to_video_results(json_path, video_output_filename, options=options)

In [ ]:
# 3. Verify JSON filenames match expected list
with open(video_output_filename, 'r') as f:
    video_results = json.load(f)
    video_filenames_set = set(video_filenames)  # <-- assumes you defined video_filenames earlier
filenames_in_video_results_set = set([im['file'] for im in video_results['images']])

for fn in filenames_in_video_results_set:
    assert fn in video_filenames_set

print("✅ Detection run, aggregation, and verification completed successfully.")

In [56]:
# Post-process results
# Run this if you want bounding boxes drawn on images
!python "C:/Users/Public/Documents/MegaDetector_App/MegaDetector/megadetector/postprocessing/postprocess_batch_results.py" \
    "{json_path}" "{detected_frame_folder_base}" --image_base_dir "{output_folder_base}" --num_images_to_sample -1


Loading results from C:/Users/Public/Documents/jsons/videos_for_MD/RR_ringtail.json
Converting results to dataframe
Finished loading MegaDetector results for 307 images from C:/Users/Public/Documents/jsons/videos_for_MD/RR_ringtail.json
Choosing default confidence threshold of 0.2 based on MD version
Finished loading and preprocessing 307 rows from detector output, predicted 254 positives.
Rendered 307 images (of 307) in 4.7 seconds (0.02 seconds per image)
Finished writing html to C:/Users/Public/Videos/videos_for_MD/video_data_detected_ringtail/detections\index.html



100%|##########| 307/307 [00:00<?, ?it/s]

100%|##########| 307/307 [00:04<00:00, 65.31it/s]


In [57]:
# Verify paths and files
print(os.path.isfile(json_path))  # Should print True if the file exists
print(os.path.isdir(output_folder_base))  # Should print True if the directory exists
print(os.path.isdir(detected_frame_folder_base))  # Should print True if the directory exists


True
True
True


In [58]:
if False:

    pass

    #%% Render all detections to videos

    from megadetector.visualization.visualize_detector_output import visualize_detector_output
    from megadetector.utils.path_utils import insert_before_extension
    from megadetector.detection.video_utils import frames_to_video

    rendering_confidence_threshold = 0.1
    target_fs = 100
    fourcc = None

    # Render detections to images
    frame_rendering_output_dir = os.path.expanduser('~/tmp/rendered-frames')
    os.makedirs(frame_rendering_output_dir,exist_ok=True)

    video_rendering_output_dir = os.path.expanduser('~/tmp/rendered-videos')
    os.makedirs(video_rendering_output_dir,exist_ok=True)

    frames_json = frame_level_output_filename

    detected_frame_files = visualize_detector_output(
        detector_output_path=frames_json,
        out_dir=frame_rendering_output_dir,
        images_dir=frame_folder_base,
        confidence_threshold=rendering_confidence_threshold,
        preserve_path_structure=True,
        output_image_width=-1)

    detected_frame_files = [s.replace('\\','/') for s in detected_frame_files]

    output_video_folder = os.path.expanduser('~/tmp/rendered-videos')
    os.makedirs(output_video_folder,exist_ok=True)

    # i_video=0; input_video_file_abs = video_filenames[i_video]
    for i_video,input_video_file_abs in enumerate(video_filenames):

        video_fs = fs_by_video[i_video]
        rendering_fs = target_fs / every_n_frames

        input_video_file_relative = os.path.relpath(input_video_file_abs,input_folder)
        video_frame_output_folder = os.path.join(frame_rendering_output_dir,input_video_file_relative)
        video_frame_output_folder = video_frame_output_folder.replace('\\','/')
        assert os.path.isdir(video_frame_output_folder), \
            'Could not find frame folder for video {}'.format(input_video_file_relative)

        # Find the corresponding rendered frame folder
        video_frame_files = [fn for fn in detected_frame_files if \
                             fn.startswith(video_frame_output_folder)]
        assert len(video_frame_files) > 0, 'Could not find rendered frames for video {}'.format(
            input_video_file_relative)

        # Select the output filename for the rendered video
        if input_folder == video_rendering_output_dir:
            video_output_file = insert_before_extension(input_video_file_abs,'annotated','_')
        else:
            video_output_file = os.path.join(video_rendering_output_dir,input_video_file_relative)

        os.makedirs(os.path.dirname(video_output_file),exist_ok=True)

        # Create the output video
        print('Rendering detections for video {} to {} at {} fps (original video {} fps)'.format(
            input_video_file_relative,video_output_file,rendering_fs,video_fs))

        frames_to_video(video_frame_files,
                        rendering_fs,
                        video_output_file,
                        codec_spec='mp4v')

    # ...for each video


    #%% Render one or more sample videos...

    # ...while we still have the frames and detections around

    ## Imports

    from megadetector.visualization import visualize_detector_output
    from megadetector.detection.video_utils import frames_to_video
    from megadetector.detection.video_utils import get_video_fs


    ## Constants and paths

    confidence_threshold = 0.2
    input_fs = 30

    frame_level_output_filename = '/a/b/c/blah_detections.filtered_rde_0.150_0.850_10_1.000.json'
    video_fn_relative = '4.10cam6/IMG_0022.MP4'
    output_video_base = os.path.expanduser('~/tmp/video_preview')


    ## Determine input frame rate

    input_video_abs = os.path.join(input_folder,video_fn_relative)
    assert os.path.isfile(input_video_abs)
    input_fs = get_video_fs(input_video_abs)


    ## Determine output frame rate

    if every_n_frames > 0:
        output_fs = input_fs / every_n_frames
    else:
        output_fs = (1.0/abs(every_n_frames))


    ## Filename handling

    video_fn_relative = video_fn_relative.replace('\\','/')
    video_fn_flat = video_fn_relative.replace('/','#')
    video_name = os.path.splitext(video_fn_flat)[0]
    output_video = os.path.join(output_video_base,'{}_detections.mp4'.format(video_name))


    rendered_detections_folder = os.path.join(output_video_base,'rendered_detections_{}'.format(video_name))
    os.makedirs(rendered_detections_folder,exist_ok=True)


    ## Find frames corresponding to this video

    with open(frame_level_output_filename,'r') as f:
        frame_results = json.load(f)

    frame_results_this_video = []

    # im = frame_results['images'][0]
    for im in frame_results['images']:
        if im['file'].replace('\\','/').startswith(video_fn_relative):
            frame_results_this_video.append(im)

    assert len(frame_results_this_video) > 0, \
        'No frame results matched {}'.format(video_fn_relative)
    print('Found {} matching frame results'.format(len(frame_results_this_video)))

    frame_results['images'] = frame_results_this_video

    frames_json = os.path.join(rendered_detections_folder,video_fn_flat + '.json')

    with open(frames_json,'w') as f:
        json.dump(frame_results,f,indent=1)


    ## Render detections on those frames

    detected_frame_files = visualize_detector_output.visualize_detector_output(
        detector_output_path=frames_json,
        out_dir=rendered_detections_folder,
        images_dir=frame_folder_base,
        confidence_threshold=confidence_threshold,
        preserve_path_structure=True,
        output_image_width=-1)


    ## Render the output video

    codec_spec = 'h264'
    # codec_spec = 'mp4v'
    frames_to_video(detected_frame_files, output_fs, output_video, codec_spec=codec_spec)

    # from megadetector.utils.path_utils import open_file; open_file(output_video)


    #%% Test a possibly-broken video

    fn = '/datadrive/tmp/video.AVI'

    fs = video_utils.get_video_fs(fn)
    print(fs)

    tmpfolder = '/home/user/tmp/frametmp'
    os.makedirs(tmpfolder,exist_ok=True)

    video_utils.video_to_frames(fn, tmpfolder, verbose=True, every_n_frames=10)


    #%% List videos in a folder

    input_folder = '/datadrive/tmp/organization/data'
    video_filenames = video_utils.find_videos(input_folder,recursive=True)


    #%% Estimate the extracted size of a folder by sampling a few videos

    n_videos_to_sample = 5

    video_filenames = video_utils.find_videos(input_folder,recursive=True)
    import random
    random.seed(0)
    sampled_videos = random.sample(video_filenames,n_videos_to_sample)
    assert len(sampled_videos) == n_videos_to_sample

    size_test_frame_folder = os.path.join(frame_folder_base,'size-test')
    if quality is not None:
        size_test_frame_folder += '_' + str(quality)
    os.makedirs(size_test_frame_folder,exist_ok=True)

    total_input_size = 0
    total_output_size = 0

    # i_video = 0; video_fn = sampled_videos[i_video]
    for i_video,video_fn in enumerate(sampled_videos):

        print('Processing video {}'.format(video_fn))
        frame_output_folder_this_video = os.path.join(size_test_frame_folder,
                                                      'video_{}'.format(str(i_video).zfill(4)))
        os.makedirs(frame_output_folder_this_video,exist_ok=True)
        video_utils.video_to_frames(video_fn,
                                    frame_output_folder_this_video,
                                    verbose=True,
                                    every_n_frames=every_n_frames,
                                    quality=quality,
                                    max_width=max_width)

        from megadetector.utils.path_utils import _get_file_size,get_file_sizes
        video_size =_get_file_size(video_fn)[1]
        assert video_size > 0
        total_input_size += video_size

        frame_size = get_file_sizes(frame_output_folder_this_video)
        frame_size = sum(frame_size.values())
        assert frame_size > 0
        total_output_size += frame_size

    import shutil # noqa
    # shutil.rmtree(size_test_frame_folder)
    import humanfriendly
    print('')
    print('Video size: {}'.format(humanfriendly.format_size(total_input_size)))
    print('Frame size: {}'.format(humanfriendly.format_size(total_output_size)))
    print('Ratio: {}'.format(total_output_size/total_input_size))





Combined video created: C:/Users/Public/Videos/RR_ringtail.AVI
